In [1]:
import nltk 
nltk.download('twitter_samples')
import nltk
nltk.download('stopwords')  #for stopwords
import nltk
nltk.download('averaged_perceptron_tagger') # for speech Tagging
import nltk
nltk.download('wordnet')  # Database of words to find meaning of a token
import nltk
nltk.download('punkt')   #converts text into list of sentences
from nltk.stem.wordnet import WordNetLemmatizer   # for lemmatizzation of a word
from nltk.corpus import twitter_samples, stopwords # for stopwords 
from nltk.tag import pos_tag                       # for tags contain noun or verb or adjective
from nltk.tokenize import word_tokenize            # for tokenization
from nltk import FreqDist, classify, NaiveBayesClassifier    #model
import re, string, random

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Functions for Cleaning the data
Tokenization, Removal of Stopwords, Lemmitization, 


In [2]:


def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)   # removing stopwords
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):                     #If a  token is NN then change its positional tag as n for noun
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()            # for generating a root word of that token
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())      # appending a cleaned token 
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token                         # it returns a generator token without exiting a end value

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)



# Gathering the data

In [3]:



positive_tweets = twitter_samples.strings('positive_tweets.json')   # positive tweets
negative_tweets = twitter_samples.strings('negative_tweets.json')   #negative tweets
text = twitter_samples.strings('tweets.20150430-223406.json')       #textfor analysis 
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0] # tokenize positive tweets

stop_words = stopwords.words('english')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json') #Tokenize positive tweets 
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json') #Tokenize negative tweets

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words)) #appends cleaned tokens without stopwords

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words)) #appends cleaned tokens without stopwords

all_pos_words = get_all_words(positive_cleaned_tokens_list)               #running this function we get all positive words in positive sent list

freq_dist_pos = FreqDist(all_pos_words)                                   # most occuring words in the list
print(freq_dist_pos.most_common(10))






[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


# Preparing data for model
Here we use naive Bayes classifier

In [4]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                        for tweet_dict in positive_tokens_for_model]                #Converting Tokens into dictionary

negative_dataset = [(tweet_dict, "Negative")
                        for tweet_dict in negative_tokens_for_model]                #Converting Tokens into dictionary

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)                                                             #Shuffling the data for Test and Train

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy:- ", classify.accuracy(classifier, test_data))

Accuracy:-  0.996


# Taking custom input for prediction

In [9]:

custom_tweet = input()

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

I ordered just once from TerribleCo, they screwed up, never used the app again
I ordered just once from TerribleCo, they screwed up, never used the app again Negative
